In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Step 1: Import data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv("../input/tabular-playground-series-mar-2022/train.csv")

test_df = pd.read_csv("../input/tabular-playground-series-mar-2022/test.csv")

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 

sns.barplot(data=train_df, x="direction", y="congestion")


In [ ]:
sns.distplot(train_df["congestion"])

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.describe()

In [ ]:
train_df['congestion'].value_counts()

In [ ]:
np.unique(train_df.congestion)

In [ ]:
train_df['time'].value_counts()

In [ ]:
#Data type conversion in the 'time' column.

train_df['time'] = train_df['time'].apply(pd.to_datetime)
train_df['year'] = train_df['time'].apply(lambda x : x.year)
train_df['month'] = train_df['time'].apply(lambda x : x.month)
train_df['day'] = train_df['time'].apply(lambda x : x.day)
train_df['hour'] = train_df['time'].apply(lambda x : x.hour)
train_df.drop(['time', 'year', 'month', 'day'], axis=1, inplace=True)

train_df

In [ ]:
test_df['time'] = test_df['time'].apply(pd.to_datetime)
test_df['year'] = test_df['time'].apply(lambda x : x.year)
test_df['month'] = test_df['time'].apply(lambda x : x.month)
test_df['day'] = test_df['time'].apply(lambda x : x.day)
test_df['hour'] = test_df['time'].apply(lambda x : x.hour)
test_df.drop(['time', 'year', 'month', 'day'], axis=1, inplace=True)

test_df

In [ ]:
np.unique(train_df.direction)

In [ ]:
# train_df['EB', 'NB', 'NE', 'NW', 'SB', 'SE', 'SW', 'WB'] = train_df['1', '2', '3', '4', '5', '6', '7', '8'].apply(lambda x: str())

# Step 2: Data preprocessing

In [ ]:
x_train = train_df.drop(['congestion','row_id'], axis=1 )
y_train = train_df['congestion']
x_test = test_df.drop(['row_id'], axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x_train['direction'] = le.fit_transform(x_train['direction'])
x_test['direction'] = le.transform(x_test['direction'])

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

sc = StandardScaler()
X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test) 

regressor = RandomForestRegressor(n_estimators=10, random_state=10)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [ ]:
y_pred

# Step 3: Predicting
RandomForest (gridsearch CV)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

reg = RandomForestRegressor()

params = {'criterion':['squared_error','friedman_mse'], 
          'max_depth': [10, 25, 50],
          'max_leaf_nodes':[10, 25, 50],
          'n_estimators': [10, 25, 50]
          }

grid = GridSearchCV(reg, 
                    params,
                    cv=5,
                    scoring = 'neg_mean_squared_error',
                    verbose=3
                    )

grid.fit(x_train, y_train)

print("GridSearch 최적 파라미터: ", grid.best_params_)
print('GridSearch 최고 점수: ', grid.best_score_)

In [ ]:
print("Best GridSearch output: ", grid.best_estimator_)

best_model = grid.best_estimator_

y_pred = best_model.predict(x_test)

In [ ]:
df_submit = pd.read_csv('./submission.csv')
df_submit['congestion'] = y_pred
df_submit.to_csv('submission.csv', index=False, header=True)

# Step 4: visualization

In [ ]:
import matplotlib.pyplot as plt

import seaborn as sns

plt.figure(figsize = (10, 10))
sns.scatterplot(x='hour', 

                y='congestion', 

                hue='group', # different colors by group

                style='group', # different shapes by group

                s=10, # marker size

                data=test_df)
 plt.show()

In [ ]:
sns.barplot(data=train_df, x="time", y="congestion")

sns.barplot(x=train_df['time'], y=train_df['congestion'], hue=train_df['hour'], dodge=False)